# Analyse d’un modèle avec demandes, stockage hydroélectrique et gaz

### Import des librairies utilisées pour résoudre les problèmes

In [1]:
import numpy as np
from fileData import FileData
import matplotlib.pyplot as plt
import cvxpy as cp

### Variables communes

In [2]:
# Couts d'installation amortis des éoliennes
cout_instal_onshore  = 168903 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)
cout_instal_offshore = 300336 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)

# Couts d'installation amortis et coûts de fonctionnement des centrales au gaz
cout_instal_gaz = 94956 # Coût *amorti sur un an* pour l'installation d'un MW de centrale au gaz (euros/MW/an)
cout_fonct_gaz  = 65    # Coût de fonctionnement pour la production d'un MWh par centrale au gaz (euros/MWh)

# Stockage disponible par pays (en MWh)
stockage = [0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6]

# Puissances maximales de turbinage et de pompage disponibles par pays (en MW)
turbinage = [8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101]
pompage   = [5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ]

# Rendement pour le turbinage (sans unité)
heta = 0.85

# Horizon temporel de résolution
"""
1 jour            : 24
1 semaine         : 168
1 mois (30 jours) : 720
3 mois (1/4 an)   : 2190
6 mois (1/2 an)   : 4380 
1 an   (365 jours): 8760
"""
h = 24

# Longueur de la période utilisée
T = 3

# Nombre de périodes
t = h//T

#Nombre d'éoliennes onshore et offshore
n_t = 487
n_m = 155
n = n_t + n_m

### Extraction des données partagées dans les modèles

In [20]:
data = FileData()

# Matrice de la consommation agrégée
D = np.sum(data.getConsoData(), axis=0)
# Rendement des éoliennes onshore et offshore
R = data.getRend()
# Capacités électriques éolien
c_t = data.getOnshoreCapacites()
c_m = data.getOffshoreCapacites()
c = np.concatenate((c_t,c_m))
# Apports naturels hydroélectriques agrégés
W = np.sum(data.getHydroData(), axis=0)

print("D", np.shape(D))
print("R", np.shape(R))
print("W", np.shape(W))

D (8760,)
R (642, 8760)
W (8760,)


In [21]:
# Réduction à l'horizon temporel considéré
D = D[:h]
R = R[:,:h]
W = W[:h]

print("D", np.shape(D))
print("R", np.shape(R))
print("W", np.shape(W))

D (24,)
R (642, 24)
W (24,)


### Fonctions communes
Fonctions communes à tous les modèles

In [22]:
def periodisation(A, T):
    """
    Additionne toutes les valeurs de A dans une même période.
    input A : Matrice à périodiser
    !! Le nombre de colonnes (axis = 1) représente le temps (l'horizon temporel) !!
    input T : Longueur de la période
    output : Matrice A périodisée
    """
    # p : nombre de période
    # r : reste du nombre de période
    p = A.shape[1] // T
    r = A.shape[1] % T
    # On coupe la matrice car sinon il n'y a pas assez de données de remplir la dernière période
    A = A[:,:-r] if r!=0 else A
    # C : matrice pour additionner les productions d'une période
    C = np.zeros((A.shape[1], p))
    for i in range(p):
        C[i * T:(i + 1) * T, i] = 1
    # P : matrice périodisée
    P = np.dot(A, C)
    return P

def E_eolienne(R, c, T):
    A = np.zeros(R.shape)
    for i in range(len(c)):
        A[i] = c[i] * R[i]
        
    A = periodisation(A, T)
    return A

### Produits des fonctions communes
Variables communes nécessitant un calcul

In [23]:
# Périodisation de D
d = periodisation(np.array([D]), T)
d = np.transpose(d)
# E : matrice de la production éolienne par site par période
E = E_eolienne(R, c, T)
# s : scalaire, stockage total hydroélectrique
s = np.sum(stockage)
# pPompage : scalaire, puissance de pompage totale
pPompage = np.sum(pompage)
# pTurbinage : scalaire, puissance de turbinage totale
pTurbinage = np.sum(turbinage)
# Périodisation de W
wA = periodisation(np.array([W]), T)
wA = np.transpose(wA)
# Matrice L
L = np.tril(np.ones((t,t)))
# Vecteur v
def v(n) :
    return(np.ones((n,1)))

print("d", np.shape(d))
print("wA", np.shape(wA))
print("L", np.shape(L))

d (8, 1)
wA (8, 1)
L (8, 8)


## Modèle 4
On considère d’abord un modèle européen agrégé avec installation d’éoliennes et utilisation des capacités hydroélectriques, mais pas de centrales au gaz. Ce modèle agrégé ne possède donc qu’une seule entité pour la production, le stockage et la consommation d’électricité (additionnez les capacités hydroélectriques et les demandes de tous les pays qui la composent).
On chercher à calculer la valeur minimale du prix moyen de l’électricité consommée.

In [24]:
x = cp.Variable((n,1))
#Variables : pompage et turbinage
wP = cp.Variable((t,1))
wT = cp.Variable((t,1))

# Vecteur des prix d'installation d'éolienne onshore et offshore
cost = (h/8760)*np.concatenate((cout_instal_onshore * c_t, cout_instal_offshore * c_m))

print(np.shape(E))
print(np.shape(wP))
print(np.shape(d))

# Contraintes
contraints = [
    0 <= x, x <= 1, 
    E.T@x - wP + wT >= d,
    0 <= s/2 + L@(wA + wP - wT/heta), s/2 + L@(wA + wP - wT/heta) <= s,
    np.transpose(v(t))@(wA + wP - wT/heta) == 0,
    0 <= wP, wP <= T*pPompage,
    0 <= wT, wT <= T*pTurbinage
    ]

# Il manque la production hydro pour satisfaire la demande donc je l'ai remplacé par la consommation d'une grosse ampoule qui éclaire h24 pendant 1 an
D = np.ones(2920)

prob = cp.Problem(cp.Minimize(cost @ x), contraints)
prob.solve()

(642, 8)
(8, 1)
(8, 1)
Le prix total optimal est :  116566700.0622546
[[9.99999997e-01]
 [9.99999999e-01]
 [5.24295689e-11]
 [4.04245663e-11]
 [3.05965701e-11]
 [3.26837835e-11]
 [8.27766129e-11]
 [5.19611341e-11]
 [5.72085046e-11]
 [6.75715858e-11]
 [5.90730149e-11]
 [5.04967923e-11]
 [5.50890985e-11]
 [6.82220817e-11]
 [7.44649176e-11]
 [6.61695252e-11]
 [6.52045738e-11]
 [7.05059906e-11]
 [7.94819520e-11]
 [1.18219702e-10]
 [1.03818750e-10]
 [8.36460402e-11]
 [7.73454171e-11]
 [8.50147512e-11]
 [8.96777457e-11]
 [8.03028224e-11]
 [8.88251421e-11]
 [1.07427520e-10]
 [1.09222113e-10]
 [1.01548286e-10]
 [9.88453004e-11]
 [8.40801084e-11]
 [9.69829843e-11]
 [8.40601050e-11]
 [7.24297395e-11]
 [7.27695410e-11]
 [1.17779908e-10]
 [8.68410139e-11]
 [1.06088527e-10]
 [7.65613779e-11]
 [5.88020824e-11]
 [6.52463718e-11]
 [1.14562229e-10]
 [1.55286348e-10]
 [1.25025321e-10]
 [5.83173211e-11]
 [4.75556512e-11]
 [2.04232340e-10]
 [1.16865543e-10]
 [1.24476635e-10]
 [1.36329134e-10]
 [1.45727129

In [25]:
# Energie produite (pour laquelle de l'argent a été dépensé)
Eprod = np.sum(E.T@x.value)

# Prix total, prix moyen
print("Prix total, {} euros".format(prob.value))
print("Prix moyen, {} euros/MWh".format(prob.value/Eprod))

print(x.value)
print()

Prix total, 116566700.0622546 euros
Prix moyen, 23.421429122530277 euros/MWh
[[9.99999997e-01]
 [9.99999999e-01]
 [5.24295689e-11]
 [4.04245663e-11]
 [3.05965701e-11]
 [3.26837835e-11]
 [8.27766129e-11]
 [5.19611341e-11]
 [5.72085046e-11]
 [6.75715858e-11]
 [5.90730149e-11]
 [5.04967923e-11]
 [5.50890985e-11]
 [6.82220817e-11]
 [7.44649176e-11]
 [6.61695252e-11]
 [6.52045738e-11]
 [7.05059906e-11]
 [7.94819520e-11]
 [1.18219702e-10]
 [1.03818750e-10]
 [8.36460402e-11]
 [7.73454171e-11]
 [8.50147512e-11]
 [8.96777457e-11]
 [8.03028224e-11]
 [8.88251421e-11]
 [1.07427520e-10]
 [1.09222113e-10]
 [1.01548286e-10]
 [9.88453004e-11]
 [8.40801084e-11]
 [9.69829843e-11]
 [8.40601050e-11]
 [7.24297395e-11]
 [7.27695410e-11]
 [1.17779908e-10]
 [8.68410139e-11]
 [1.06088527e-10]
 [7.65613779e-11]
 [5.88020824e-11]
 [6.52463718e-11]
 [1.14562229e-10]
 [1.55286348e-10]
 [1.25025321e-10]
 [5.83173211e-11]
 [4.75556512e-11]
 [2.04232340e-10]
 [1.16865543e-10]
 [1.24476635e-10]
 [1.36329134e-10]
 [1.4

## Modèle 5
Considérons le modèle 4 avec la contrainte supplémentaire suivante : sur chaque site  éolien potentiel, il faut choisir entre installer 100%, 50% ou 0% de la capacité maximale installable.

In [ ]:
# TODO

## Modèle 6
Revenons au modèle initial de la question 4 et incorporons la possibilité d’installer et d’utiliser des centrales au gaz.

In [ ]:
# TODO